# Automated Bed Assignments in a Complex and Dynamic Hospital Environment

This assignment uses hospital data related to Royal United Hospital Bath. Please see some of the main resources used in the references listed at the bottom.

The two main aspects when creating the data include the hospital and patient data. Below are the variables that corespond to each of the sets of data. 
1. Hospital Data:
  * **departments**: List of department names (27 in total)
  * **units**: List of unit names (31 in total)
  * **rooms**: List of rooms. Each room is given a name in the format: "R" + number (ex. 'R0' is the first room)
  * **beds**: List of beds. Each bed is given a name in the format: "B" + number (ex. 'B1' is the second bed)
  * **unit_of_bed**: List of the unit name that corresponds to each bed. The list is ordered by `beds`.
  * **beds_in_unit**: List of beds, by name, in each unit.
  * **beds_in_room**: List of beds, by name, in each room
  * **bed_type**: Binary list of beds by type ("S", "B", "M", "F", "I", "G"). 
     * Specialty Bed: Can support additional needs that most patients in a unit do not need. Note, if a specialty patient supersedes gender (ie Male or Female patient). 
     * Baby Bed: Can only support infants  
     * Male: Can support Male Patient
     * Female: Can support Female Patient
     * Isolation Requirement: States whether a bed can support patient medical isolation requirements. 
     * Gender Request: States whether a bed is in a single gendered room. 
  * **tier**: Array of unit to unit. It compares unit compatability.
     * Tier 1: Unit A is the same unit as Unit B 
     * Tier 2: Unit A is in the same department as Unit B
     * Tier 3: Unit A is in the same zone as Unit B
     * Tier 4: Unit A is not the same zone as Unit B <br><br>
2. Patient Data:
  * **patients**: List of patients. Each patient is given a name in the format: "P" + number (ex. 'P5' is the sixth patient)
  * **patients_of_unit**: List of patients, by name, in each unit.
  * **patients_of_type**: Binary list of patients by type ("S", "B", "M", "F", "I", "G").
     * Specialty Patient: Patient who has additional needs that cannot be supported by "typical" patients. 
     * Baby Bed: Patient is a baby   
     * Male: Patient is Male
     * Female: Patient is Female
     * Isolation Requirement: Patient medically requires isolation. 
     * Gender Request: Patient does or does not request to be in a single gendered room. 
  * **previous_assignemnt**: Binary list of all patients and their bed assignment. For example, an individual patient's assignment will include all beds in the hospital as 0s. If that patient is already in a bed then that bed will be a 1. (ex. if there were only 4 beds in the hospital and patient i is assigned then: [0,0,1,0])


---

To quickly modify patient data, change the `percent_new` variable below. This variable is the percent of incoming patients that were not previously assigned. For the purpose of this model, we examined increases in patient size with 8%, 15%, and 25%. 

In [1]:
percent_new = 0.08

Below is the code used to create each of these variables

In [2]:
#import needed libraries
import pandas as pd
import numpy as np
import copy
import random
import math

import warnings
warnings.filterwarnings("ignore") 

# Hospital Data Creation:
---

<br>

### Create Dataframe of Unit and Department Binary 

In [3]:
dept_ward = pd.read_excel (r'Hospital Comparison Data.xlsx','Dept_Unit',
                           index_col = 0,usecols='A:C', nrows=31)
dept_ward.head()

#create a matrix of unit by department with binary as to whether or not unit is in department
departments = dept_ward['Department']
wards = dept_ward.index

#change column and row names to departments and wards respectively 
dep_ward_binary = pd.DataFrame(np.zeros((len(dept_ward.index),len(departments.unique()))))
dep_ward_binary.index = wards
dep_ward_binary.columns = departments.unique()
dep_ward_binary

#change 0s to 1s when unit, u, is in department, d
for i in range(0,len(wards)):
    dep_ward_binary.loc[wards[i],departments[i]] = 1

print("There are", len(departments.unique()), "departments and", len(wards), "units.", )
print("Below is the array of binaries for units per department:")
dep_ward_binary.head()

There are 27 departments and 31 units.
Below is the array of binaries for units per department:


,B6,B45,D9,D8,B1,B19,C1,Emergency,B34,B50,...,D1,B52,B41,B54,B40,B47,B43,C30,B55,A5
Inpatient Wards/Units,,,,,,,,,,,,,,,,,,,,,
Acute Stroke Unit,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cardiac Ward,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Charlotte Ward,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cheselden Ward,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Children's Ward,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<br>

### Create Needed Data for Beds per Ward/Unit 

The code below imports sxcel file and modifies it to fill null value using mean imputation

In [4]:
bed_per_ward = pd.read_excel (r'Hospital Comparison Data.xlsx','Ward_Bed_Specialty',
                              index_col = 0,usecols='A:G', nrows=31)

#mean imputation and remove NA values:
mean_bed_count = bed_per_ward['TOTAL'].mean(skipna=True) #.sum(skipna=True)

bed_per_ward.loc[bed_per_ward['S'].isnull(), 'S'] = 0 #Specialty
bed_per_ward.loc[bed_per_ward['B'].isnull(), 'B'] = 0 #Baby
bed_per_ward.loc[bed_per_ward['M'].isnull(), 'M'] = 0 #Male
bed_per_ward.loc[bed_per_ward['F'].isnull(), 'F'] = 0 #Female

bed_per_ward.loc[bed_per_ward['G'].isnull(),'G'] = mean_bed_count   #Assume all beds of unknown wards are capable of full service
bed_per_ward.loc[bed_per_ward['TOTAL'].isnull(),'TOTAL'] = mean_bed_count

### Create a list based on nurse ratios

In [5]:
units = list(dept_ward.index)


#Assume all units are a 1:8 ratio or nurses to patients 
#except for ICU units which maintain a 1:1
nurse_ratio= [1/8]*len(units)

#units.index("Intensive Care Unit")
for unit_i in units:
    if "Intensive Care Unit" in unit_i:
        nurse_ratio[units.index(unit_i)] = 1
        
print(nurse_ratio)

[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 1, 1, 0.125, 0.125, 0.125, 0.125, 0.125, 1, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]


In [6]:
max_patients_per_unit = list(bed_per_ward["TOTAL"])
max_patients_per_unit = [int(x) for x in max_patients_per_unit] 
print(max_patients_per_unit)

[26, 28, 22, 22, 25, 26, 8, 25, 25, 36, 28, 25, 14, 14, 25, 39, 33, 18, 25, 21, 27, 28, 28, 30, 30, 33, 24, 25, 19, 24, 22]


<br>

### Create List of Beds

In [7]:
#calculate the number of bed 
beds = []
for i in range(0,int(sum(bed_per_ward['TOTAL']))):
    bed = "B" + str(i)
    beds.append(bed)
    
print("beds: ", beds[:5], "...",beds[-5:])

beds:  ['B0', 'B1', 'B2', 'B3', 'B4'] ... ['B770', 'B771', 'B772', 'B773', 'B774']


<br>

### Create List of Beds in each Unit

In [8]:
#calculate the number of beds_in_unit (with sub-list format)
bed_count = list(bed_per_ward['TOTAL'])

beds_in_unit = []

index = 0
for bed in bed_count:
    if index==0:
        beds_in_unit.append(beds[0:int(bed)])
        index = int(bed)
        
    beds_in_unit.append(beds[index:int(bed)+index])
    index += int(bed)

beds_in_unit.pop() #remove empty list
print(beds_in_unit[0])

['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25']


<br>

### Create List of Units in each bed

In [9]:
unit_index_val = 0
unit_of_bed = []

for bed_i in beds_in_unit:
    unit_of_bed.append(len(bed_i)* [units[unit_index_val]])
    unit_index_val+=1


unit_of_bed = sum(unit_of_bed,[])
print(unit_of_bed[:20])

['Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit', 'Acute Stroke Unit']


### Create Needed Data for Beds per Room for each Ward/Unit

In [10]:
bed_per_room_for_ward = pd.read_excel (r'Hospital Comparison Data.xlsx','Unit_Room_Bed_Count', 
                                       header = 0, index_col = 0, nrows=31)

#determine which columns have missing data
missing_ward_data = bed_per_room_for_ward[bed_per_room_for_ward['R1'].isnull()==True].index

for row, index in bed_per_room_for_ward.iterrows():
    if row in missing_ward_data: #for missing data, fill R1 to R25 with single bed rooms, and the rest with 0s     
        fill_data_1 = list(index.index)[:25] 
        index[fill_data_1] = 1
        
        fill_data_0 = list(index.index)[25:] #nonexistant rooms
        index[fill_data_0] = 0


bed_per_room_for_ward.head(2)

,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,...,R24,R25,R26,R27,R28,R29,R30,R31,R32,R33
Inpatient Wards/Units,,,,,,,,,,,,,,,,,,,,,
Acute Stroke Unit,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cardiac Ward,6.0,6.0,6.0,6.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<br>

### Calculate Number of Rooms 

In [11]:
#Calculate number of rooms
total_rooms = bed_per_room_for_ward[bed_per_room_for_ward>0.0].count().sum() #counts rows with values that have at least 1 bed and sums values

rooms = []
for i in range(0,total_rooms):
    room = "R" + str(i)
    rooms.append(room)

print(rooms[:5],"...",rooms[-5:])

['R0', 'R1', 'R2', 'R3', 'R4'] ... ['R539', 'R540', 'R541', 'R542', 'R543']


<br>

### Create list of bed types

In [12]:
bedtype_per_room_for_ward = pd.read_excel (r'Hospital Comparison Data.xlsx','bed_type', 
                                           header = 0, index_col = 0, nrows=31)

#Determine SPECIFIC bed type for each bed (S,B,M,F)
bed_type_i = []
for i, j in zip(sum(bed_per_room_for_ward.values.tolist(),[]), sum(bedtype_per_room_for_ward.values.tolist(),[])):
    bed_type_i.append(int(i)*j)
    
bed_type_i= [i for i in bed_type_i if i != 0]
bed_type_i= [i for i in bed_type_i if i != ""]
bed_type_i = list(''.join(bed_type_i))         #NOTE List of G,S, or B

#===================================================================================#
#Make nested list of beds with binaries of bed type --> [S B M F] -->ex [1 0 0 0]
random.seed(1)
bed_type = [] #[S B M F]

for bed_i in range(0,len(beds)):
    if bed_type_i[bed_i] == "G": #General cases 
        #gender_prob = random.randrange(2)   #50% probability of being male or female
        bed_type.append([0, 0, 1, 1])
    
    elif bed_type_i[bed_i] == "S": #Special cases 
        bed_type.append([1, 0, 0, 0])
    
    elif bed_type_i[bed_i] == "B": #Baby cases 
        bed_type.append([0, 1, 0, 0])
        
    elif bed_type_i[bed_i] == "M": #Male cases 
        bed_type.append([0, 0, 1, 0])
        
    else: #Female cases
        bed_type.append([0, 0, 0, 1])
        
#SPECIFY Individual or Group Room
bed_per_room = []
for bed_count in [i for i in sum(bed_per_room_for_ward.values.tolist(), []) if i != 0]:
    if int(bed_count) == 1:
        bed_per_room.append(["S"]) #for single
        
    else:
        bed_per_room.append(["M"]*int(bed_count)) #for multiple
    
bed_per_room = sum(bed_per_room,[])

#===================================================================================#
#Append to list - Individual bed required
for bed_i in range(0,len(beds)):
    if bed_per_room[bed_i] == "S": #single room 
        bed_type[bed_i].append(1)
    
    else: #if more than one bed 
        bed_type[bed_i].append(0) #multple room
        
#===================================================================================#
#Append to list - Gender bed requirment 
for bed_i in range(0,len(beds)):
    if bed_per_room[bed_i] == "S": #single room 
        bed_type[bed_i].append(1)
    
    else: #if more than one bed 
        if bed_type_i[bed_i]=="M" or bed_type_i[bed_i]=="F": #and the room is gendered
            bed_type[bed_i].append(1) #multple room 
        else:
            bed_type[bed_i].append(0) #multple non-gendered room
        
bed_type_df = pd.DataFrame(bed_type, columns=["S","B","M","F","I","G"], index=beds)
bed_type_df.head(3)

,S,B,M,F,I,G
B0,0,0,1,1,1,1
B1,0,0,1,1,1,1
B2,0,0,1,1,1,1


In [13]:
bed_type_df

,S,B,M,F,I,G
B0,0,0,1,1,1,1
B1,0,0,1,1,1,1
B2,0,0,1,1,1,1
B3,0,0,1,1,1,1
B4,0,0,1,1,1,1
...,...,...,...,...,...,...
B770,1,0,0,0,1,1
B771,1,0,0,0,1,1
B772,1,0,0,0,1,1
B773,1,0,0,0,1,1


<br>

### Determine the bed name in each room

In [14]:
#create lists number of beds in each room
bed_count_room = []
for unit in list(bed_per_room_for_ward.index):
    fs = [int(x) for x in list(bed_per_room_for_ward.loc[unit])]
    fs = [x for x in fs if x>0]
    bed_count_room.append(fs)

    
#Create matrix of bed(s) that are in each room 
beds_in_room = []

index = 0
for bed in sum(bed_count_room,[]):
    if index==0:
        beds_in_room.append(beds[0:int(bed)])
        index = int(bed)
        
    beds_in_room.append(beds[index:int(bed)+index])
    index += int(bed)

print(beds_in_room[:25]) #subsection 

[['B0'], ['B1'], ['B2'], ['B3'], ['B4'], ['B5'], ['B6'], ['B7'], ['B8'], ['B9'], ['B10'], ['B11'], ['B12'], ['B13'], ['B14'], ['B15'], ['B16'], ['B17'], ['B18'], ['B19'], ['B20'], ['B21'], ['B22'], ['B23', 'B24', 'B25', 'B26'], ['B27', 'B28', 'B29', 'B30', 'B31', 'B32']]


<br>

### Create Bed Tier Data


In [15]:
#Make tiers based off of the units
unit_tiers = []
for unit_i in units: #for each unit
    unit_i_tier = []
    
    for location in units: #for each department
        if unit_i == location: #If they are in the same unit
            unit_i_tier.append(1)
            
        elif dept_ward.loc[unit_i]["Department"]== dept_ward.loc[location]["Department"]: #If they are in the same department
            unit_i_tier.append(2)
        
        elif dept_ward.loc[unit_i]["Zone"]== dept_ward.loc[location]["Zone"]: #If they are in the same zone
            unit_i_tier.append(3)
        
        else:
            unit_i_tier.append(4)
    unit_tiers.append(unit_i_tier)

pd.DataFrame(unit_tiers, index=units, columns=units).head(3)

,Acute Stroke Unit,Cardiac Ward,Charlotte Ward,Cheselden Ward,Children's Ward,Comb Ward,Coronary Care Unit,Direct Assessment Area,ED Observation Ward,Forrester Brown Ward,...,Parry Ward,Phillip Yeoman Ward,Pierce Ward,Pulteney Ward,Respiratory Unit,Robin Smith Ward,Stroke Unit,Surgical Assessment Unit (SAU),Waterhouse Ward,William Budd Ward
Acute Stroke Unit,1,3,4,4,3,3,3,4,4,3,...,3,3,3,3,3,3,2,4,3,4
Cardiac Ward,3,1,4,4,3,3,2,4,4,3,...,3,3,3,3,3,3,3,4,3,4
Charlotte Ward,4,4,1,3,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [16]:
#---------ROOM COLUMN----------#
room_index=0
room_for_bed_i=[]

for room_i in beds_in_room:
    if len(room_i)==1:
        room_for_bed_i.append(("R"+str(room_index)))
        room_index+=1
    else:
        for bed_i in room_i:
            room_for_bed_i.append(("R"+str(room_index)))
        room_index+=1
        
#print(room_for_bed_i)

#---------UNIT COLUMN----------#
unit_index = 0
beds_in_unit_list = []
for unit_i in beds_in_unit:
    for bed_i in unit_i:
        beds_in_unit_list.append(dept_ward.iloc[unit_index].name)
    unit_index+=1
    


#---------BED_TYPE COLUMN----------#
#bed_type
hospital_data = pd.DataFrame(list(zip(beds, bed_type,room_for_bed_i,beds_in_unit_list)), 
                             columns=["bed","bed_type","room","unit"])
hospital_data.head()

,bed,bed_type,room,unit
0,B0,"[0, 0, 1, 1, 1, 1]",R0,Acute Stroke Unit
1,B1,"[0, 0, 1, 1, 1, 1]",R1,Acute Stroke Unit
2,B2,"[0, 0, 1, 1, 1, 1]",R2,Acute Stroke Unit
3,B3,"[0, 0, 1, 1, 1, 1]",R3,Acute Stroke Unit
4,B4,"[0, 0, 1, 1, 1, 1]",R4,Acute Stroke Unit


<br><br><br><br><br><br><br><br><br>

---

# Patient Data Creation:

<br>

#### Import dataset

In [17]:
#Import
night_patients = pd.read_excel (r'Hospital Comparison Data.xlsx','Night_Occupation', 
                                       header = 10, usecols='B:AA', nrows=4)

#remove unneeded rows 
night_patients = night_patients.drop(columns=["Quarter","SHA Code", "Org Code","Org Name","Unnamed: 6"])

#Calculate mean totals
night_patients.loc["TOTALS"] = round(night_patients.mean(),0)


<br>

### Create Patient List

Note: Average number of patients + 10% (who will enter the hospital)

In [18]:
#Determine average expected number of patients at midnight
patients_time_0 = int(night_patients.loc["TOTALS"].sum())
new_patients_time_0 = int(round(percent_new *patients_time_0,0))

#create patient list
patients = []
for i in range(0, patients_time_0 + new_patients_time_0): #patients + 10% of patients at t=0
    patient = "P" + str(i)
    patients.append(patient)

print("patients: ", patients[:5], "...", patients[-5:])

patients:  ['P0', 'P1', 'P2', 'P3', 'P4'] ... ['P614', 'P615', 'P616', 'P617', 'P618']


<br><br>

### Create Dataframe of Patient Units

In [19]:
#FOR UNASSIGNED PATIENTS
#Ensure data stays same
random.seed(2)

#calculate probability of a patient being in a unit
units = list(dept_ward.index)
unit_prob = [int(x) for x in list(bed_per_ward['TOTAL'])]
unit_prob = [(x/sum(unit_prob)) for x in unit_prob]

#Determine unit information for "new patients"
patient_unit = []
patient_unit_index=[]

for i in range(len(patients)):
    #pick patient based on ratio of beds in a unit
    item = random.choices(units, weights=unit_prob)
    
    #determine the index of unit
    unit_index = [i for i, e in enumerate(units) if e == item[0]][0]
    patient_unit_index.append(unit_index)
    
    #append information to binary list for each patient
    pat_i_unit = [0]*len(units) #list of 0s
    pat_i_unit[unit_index] = 1  #change unit patient is in to 1
    
    patient_unit.append(pat_i_unit)
    
#convert to df
patient_units= pd.DataFrame(patient_unit, columns=units, index=patients)
patient_units.head()

,Acute Stroke Unit,Cardiac Ward,Charlotte Ward,Cheselden Ward,Children's Ward,Comb Ward,Coronary Care Unit,Direct Assessment Area,ED Observation Ward,Forrester Brown Ward,...,Parry Ward,Phillip Yeoman Ward,Pierce Ward,Pulteney Ward,Respiratory Unit,Robin Smith Ward,Stroke Unit,Surgical Assessment Unit (SAU),Waterhouse Ward,William Budd Ward
P0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
P1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
P2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [20]:
patients_of_unit=[]

for unit_i in units:
    patients_of_unit.append(list((patient_units.loc[patient_units[unit_i] == 1]).index))

print(patients_of_unit[1])

['P2', 'P20', 'P87', 'P115', 'P118', 'P151', 'P152', 'P171', 'P191', 'P264', 'P486', 'P513', 'P532', 'P539', 'P552', 'P592', 'P597']


<br><br><br><br><br><br>

### Create Dataframe of patient types

In [21]:
#Ensure data stays same
random.seed(4)

patients_type = []

for unit_i in patient_unit: # for each patient's unit
    unit_index = ([i for i, e in enumerate(unit_i) if e == 1][0])
        
    bed_type_u = bed_per_ward.loc[units[unit_index],:]
    
    #Calculate probabilities
    s_prob = bed_type_u[1] #specialty
    b_prob = bed_type_u[2] #baby
    m_prob = (bed_type_u[0]/2)+ bed_type_u[3] #male
    f_prob = (bed_type_u[0]/2)+ bed_type_u[4] #female
    
    unit_prob = [s_prob, b_prob, m_prob, f_prob]
    
    #use probabilities to determine patient type
    item = random.choices(([1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]), weights=unit_prob)
    
    patients_type.append(sum(item,[]))
    
#======================================================================
#ADD I and G requirements / requests
for pat_i in patients_type:
    
    #add isolation and gender requirment for specialty and baby (ie no gender nor isolation)
    if pat_i[0]==1 or pat_i[1]==1: #if baby or specialty - no isolation can be medically required, nor gender requested
        pat_i.append(0)
        pat_i.append(0)
    
    else:
        #Calculate probabilities
        #NOTE: Assume that 10% of patients will be medically required to be isolated        
        #      Assume that 60% of patients will request a gendered room
        
        #use probabilities to determine patient requirements/requests
        isolation_request = random.choices((1,0), weights=[0.1,0.9])[0]
        gender_request = random.choices((1,0), weights=[0.6,0.4])[0]
        
        pat_i.append(isolation_request)
        pat_i.append(gender_request)

In [22]:
patient_type = pd.DataFrame(patients_type, columns=["S", "B", "M", "F","I","G"], index=patients)
patient_type.head(3)

,S,B,M,F,I,G
P0,0,0,1,0,0,0
P1,0,0,1,0,0,1
P2,0,0,1,0,0,0


In [23]:
patients_of_type=[]

for type_i in ["S", "B", "M", "F","I","G"]:
    patients_of_type.append(list((patient_type.loc[patient_type[type_i] == 1]).index))

print(patients_of_type[1])

['P8', 'P9', 'P80', 'P120', 'P146', 'P164', 'P200', 'P311', 'P325', 'P337', 'P445', 'P482', 'P496', 'P501', 'P555', 'P588']


### Combine patient data

In [24]:
#combine patient information
patient_data = pd.DataFrame(list(zip(patient_unit_index, patients_type)), columns=["unit","type"], index=patients)
patient_data.head()

,unit,type
P0,29,"[0, 0, 1, 0, 0, 0]"
P1,29,"[0, 0, 1, 0, 0, 1]"
P2,1,"[0, 0, 1, 0, 0, 0]"
P3,2,"[0, 0, 0, 1, 0, 0]"
P4,25,"[1, 0, 0, 0, 0, 0]"


<br><br><br><br><br><br>

---
## INFROMATION NEEDED FOR PREVIOUS ASSIGNMENT .DAT FILE

In [25]:
patients_0_of_unit=[]

for unit_i in units:
    patients_0_of_unit.append(list((patient_units[:patients_time_0].loc[patient_units[:patients_time_0][unit_i] == 1]).index))

print(patients_0_of_unit[1])

['P2', 'P20', 'P87', 'P115', 'P118', 'P151', 'P152', 'P171', 'P191', 'P264', 'P486', 'P513', 'P532', 'P539', 'P552']


In [26]:
patients_0_of_type=[]

for type_i in ["S", "B", "M", "F","I","G"]:
    patients_0_of_type.append(list((patient_type[:patients_time_0].loc[patient_type[type_i] == 1]).index))

print(patients_0_of_type[1])

['P8', 'P9', 'P80', 'P120', 'P146', 'P164', 'P200', 'P311', 'P325', 'P337', 'P445', 'P482', 'P496', 'P501', 'P555']


In [27]:
with open('TAOR_Data_Previous_Assignment.txt', 'w') as f:
    
    f.write("!Original Patient Data : \n\n")
    
    p_old_names = ["patients: ", "patients_of_unit: ", "patients_of_type: "]
    p_old_data = [patients[:patients_time_0], patients_0_of_unit, patients_0_of_type]
    p_old_result = zip(p_old_names, p_old_data)
    
    #units
    for var, data in p_old_result:
        f.write(var + str(data)+ "\n\n")

Note, the above information was used to find what bed the original 573 patients needed to be assigned to. 

The file imported below is what the model output for bed locations. It will be needed for patient's previous_assignment. 

In [28]:
assigned_beds = pd.read_csv(r'previous_assignments.csv', header=None, names=["patient","bed"])
assigned_beds.head(3)

,patient,bed
0,P0,B748
1,P1,B751
2,P2,B51


**Note**: the way the csv file was created, there are random spaces throughout patient abd bed columns. for this reason we must remove the spaces

In [29]:
old_previous_assignemnt=[]
for pat_i in range(patients_time_0):
    #location of bed
    pat_i_bed = [0] * len(beds)
    pat_i_bed[int((assigned_beds["bed"][pat_i].strip()).split("B")[1])] = 1
    
    old_previous_assignemnt.append(pat_i_bed)

new_previous_assignemnt= [[0]*len(beds)] * new_patients_time_0 

#combine assignments
previous_assignemnt = old_previous_assignemnt+new_previous_assignemnt

Now that we have previous assignment, we can utilize the last of our data

---
## INFROMATION NEEDED FOR .DAT FILE

In [30]:
#Hospital Variables
departments = dept_ward['Department'].unique() #names of inpatient departments
units = list(dept_ward.index) #names of units
dep_unit = np.asarray(dep_ward_binary) #binaries of whether a unit is in a department

In [31]:
#note, mosel file does not need commas and, depending on the dataset, does not need brackets either
with open('TAOR_Data_8%.txt', 'w') as f:
    
    f.write("!Hospital Data: \n\n")
    
    h_names = ["departments: ", "units: ", "rooms: ", "beds: ", 
               "unit_of_bed: ","beds_in_unit: ","beds_in_room: ", "bed_type: ", "tier: "]
    h_data = [str(departments).replace(",",""), str(units).replace(",",""), str(rooms).replace(",",""), 
              str(beds).replace(",",""), str(unit_of_bed).replace(",",""), str(beds_in_unit).replace(",",""), 
              str(beds_in_room).replace(",",""), str(bed_type).replace(",",""), str(unit_tiers).replace(",","")]
    h_result = zip(h_names, h_data)
    
    #nurse data
    f.write("nurse_ratio: " + str(nurse_ratio) + "\n\n\n")
    f.write("max_patients_per_unit: " + str(max_patients_per_unit) + "\n\n\n")
    
    #units
    for var, data in h_result:
        f.write(var + str(data)+ "\n\n\n")

    
    f.write("\n\n\n!---------------------------------- \n\n!Patient Data: \n\n")
    
    p_names = ["patients: ", "patients_of_unit: ", "patients_of_type: ", "previous_assignemnt"]
    p_data = [str(patients).replace(",",""), str(patients_of_unit).replace(",",""), 
              str(patients_of_type).replace(",",""), str(previous_assignemnt).replace(",","")]
    p_result = zip(p_names, p_data)
    
    #units
    for var, data in p_result:
        f.write(var + str(data)+ "\n\n")